# mwb_flow.prep Package Example 1.1
This example is update to the prep_Example1_PullData.ipynd and serves three purposes. First, it shows a workflow for creating a dataset of monthly precip and temperature data that can be used as an input for the mwb_flow model. Second, it compares three methods of retriving GridMet data and calculating zonal stats for each polygon. Lastly, it contains code for each proposed method that can be refrenced if needed later.  

In [2]:
# More inports than nessasary...
import os
from pathlib import Path
import geopandas as gpd
import xarray as xr

# Block of imports needed for GRIDtool.grid_area_weighted_volume
import rasterio as rio
import pandas as pd
from chmdata.thredds import GridMet, BBox
import numpy as np
from shapely.geometry import Polygon



mwb_flow_dir = r'C:\Users\CND905\Downloaded_Programs\mwb_flow'
os.chdir(mwb_flow_dir)

from prep.datafile import CreateInputFile
from prep.metdata import get_gridmet_at_points, get_gridmet_for_polygons, get_gridmet_for_polygons_with_xvec
from prep.datafile import check_format

import py3dep
from tqdm import tqdm
from prep.utils import get_gridmet_cells
from config import GRIDMET_PARAMS
import GRIDtools as gt
import rioxarray
import xvec 
import shapely

from geocube.api.core import make_geocube



Initializing mwb_flow.prep module.


## Load in Shape File
Import a shape file with deliniated watershed polygon. This shape file has an attribute table with a column used to index the geometries. In this case, a column with the gage station numbers was used. 

In [3]:
exres_pth = Path(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\Examples\data\Lolo_WB_Model_Calibration_Catchments_32611.shp')
exres = gpd.read_file(exres_pth)
exres = exres.to_crs(4326) # This file is in crs 32611 (WGS84 UTM zone 11N), need it to be 4326 for getting GridMET.

## Pull GridMet Data and Perform Zonal Stats
Three functions from prep.metdata.py are shown below. The first two functions, 'get_gridmet_for_polygons()' and 'get_gridmet_for_polygons_xvec()' pull meterologic data and perform area zonal stats (area weighted for precip and mean for temperature parameters) for polygons but use diffrent packages to do so. More details for each function below.

The last function 'get_gridmet_at_points()' shows the original function that pulls meterologic data but only calculates the mean for polygons. The function is slow. 

### get_gridmet_for_polygons()

This function uses the geocube package which rasterizes all of the polygons at once. <ins>This function appears to be the fastest so we will use it for the rest of the workflow.</ins>

Further testing of speed with larger range of dates, polygons and parameters to come. 

In [ ]:
exres_met = get_gridmet_for_polygons(exres, "gageID", start='2016-08-01', end='2016-11-30')
exres_met

<xarray.Dataset> Size: 5kB
Dimensions:        (location: 6, time: 32)
Coordinates:
  * location       (location) int64 48B 1000 2800 3000 5500 9500 9600
  * time           (time) datetime64[ns] 256B 2016-01-01 ... 2016-02-01
    lat            (location) float64 48B 46.7 46.82 46.78 46.67 46.75 46.75
Data variables:
    max_temp       (time, location) float64 2kB 264.8 265.4 ... 271.9 272.3
    min_temp       (time, location) float64 2kB 255.3 255.9 ... 265.4 265.7
    precip_volume  (time, location) float64 2kB 2.952e+07 0.0 ... 1.172e+08
Attributes:
    featureType:  timeSeries

### get_gridmet_for_polygons_with_xvec()

This function uses the xvec package to calculate the mean of non-area weighted parameters. The iterate method argument was used to perform zonal statistics which is a slower method than the 'rasterize' method in the package. This option has the potential to be fast if bugs in the 'rasterize' method are fixed in the package.

In [ ]:
get_gridmet_for_polygons_with_xvec(exres, "gageID", start='2016-08-01', end='2016-11-30')

<xarray.Dataset> Size: 5kB
Dimensions:        (location: 6, time: 32)
Coordinates:
    lat            (location) float64 48B 46.82 46.78 46.75 46.75 46.7 46.67
  * location       (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time           (time) datetime64[ns] 256B 2016-01-01 ... 2016-02-01
Data variables:
    max_temp       (location, time) float64 2kB 265.4 268.8 ... 270.7 270.4
    min_temp       (location, time) float64 2kB 255.9 256.8 ... 264.8 263.8
    precip_volume  (location, time) float64 2kB 0.0 0.0 ... 5.445e+08 2.578e+08
Attributes:
    featureType:  timeSeries

### get_gridmet_at_points()
This function does not calculate the area weighted volume for precipitation that the other two function do but we can run this function just to comapre the speed with the other two fuctions.

In [ ]:
get_gridmet_at_points(exres, 'gageID', start='2016-08-01', end='2016-11-30')

C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\metdata.py:55: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))


Retrieving GridMET cells...


C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\metdata.py:76: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid


70 unique GridMET cells found for 6 input features.
Fetching GridMET data for unique cells...


Cells: 100%|██████████| 70/70 [02:51<00:00,  2.44s/it]


<xarray.Dataset> Size: 5kB
Dimensions:   (time: 32, location: 6)
Coordinates:
    lat       (location) float64 48B 46.82 46.78 46.75 46.75 46.69 46.69
    lon       (location) float64 48B -114.4 -114.4 -114.2 -114.2 -114.5 -114.3
    elev      (location) float64 48B 1.435e+03 1.229e+03 ... 1.402e+03 1.741e+03
  * location  (location) int64 48B 2800 3000 9600 9500 1000 5500
  * time      (time) datetime64[ns] 256B 2016-01-01 2016-01-02 ... 2016-02-01
Data variables:
    precip    (time, location) float64 2kB 0.0 0.0 0.0 0.0 ... 5.562 4.064 5.204
    min_temp  (time, location) float64 2kB 255.8 255.4 256.5 ... 264.3 263.5
    max_temp  (time, location) float64 2kB 265.9 265.1 265.4 ... 271.4 270.3
Attributes:
    featureType:  timeSeries

## Format Dataset to Use with MWB_Flow model

Calculate the monthly average temperature.

In [7]:
# Calculate daily mean air temperature then followed by the monthly mean air temperature
mean_temp = ((exres_met.min_temp + exres_met.max_temp) / 2) - 273.15  # also convert to Celcius from GridMET native Kelvin
mean_temp
monthly_temp = mean_temp.resample(time = "MS").mean()

# Convert to a DataArray with attributes and title
Monthly_Temp = xr.DataArray(monthly_temp, coords=monthly_temp.coords, attrs={'standard_name': 'Monthly_Temperature', 'units': 'Celcius'})
Monthly_Temp.name = 'mo_temp'
Monthly_Temp

<xarray.DataArray 'mo_temp' (time: 2, location: 6)> Size: 96B
array([[-3.11687345, -2.70322581, -2.73560794, -3.3437276 , -2.56693548,
        -2.88924731],
       [-5.23076923, -4.7       , -4.46153846, -6.08888889, -4.53125   ,
        -4.15      ]])
Coordinates:
  * location  (location) int64 48B 1000 2800 3000 5500 9500 9600
    lat       (location) float64 48B 46.7 46.82 46.78 46.67 46.75 46.75
  * time      (time) datetime64[ns] 16B 2016-01-01 2016-02-01
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

Calculate the montly area weighted precipitation volume.

In [8]:
# Calculate the monthly precipitation
monthly_precip = exres_met.precip_volume.resample(time = "MS").sum()

# Convert to a DataArray with attributes and title
Monthly_Precip = xr.DataArray(monthly_precip, coords=monthly_precip.coords, attrs={'standard_name': 'Monthy Area Weighted Precipitation Volume', 'units': 'm^3'})
Monthly_Precip.name = 'mo_precip'
Monthly_Precip

<xarray.DataArray 'mo_precip' (time: 2, location: 6)> Size: 96B
array([[2.21130521e+10, 6.32413645e+09, 1.83323137e+10, 1.55141152e+10,
        1.55534657e+10, 8.61503505e+09],
       [3.63941646e+08, 7.79564001e+07, 3.26146471e+08, 2.57775092e+08,
        2.33840587e+08, 1.17175972e+08]])
Coordinates:
  * location  (location) int64 48B 1000 2800 3000 5500 9500 9600
    lat       (location) float64 48B 46.7 46.82 46.78 46.67 46.75 46.75
  * time      (time) datetime64[ns] 16B 2016-01-01 2016-02-01
Attributes:
    standard_name:  Monthy Area Weighted Precipitation Volume
    units:          m^3

In [9]:
Monthly_Q = xr.load_dataarray(r'C:\Users\CND905\Downloaded_Programs\mwb_flow\prep\q_datafile_output.nc')
Monthly_Q

<xarray.DataArray 'mo_discharge' (time: 4, location: 6)> Size: 192B
array([[0.5593601 , 0.0408128 , 0.6090988 , 0.60482234, 0.98153449,
        0.12890484],
       [0.58416357, 0.03919744, 0.70052005, 0.42791986, 1.19420146,
        0.25669362],
       [1.15964166, 0.1106497 , 1.45650749, 1.23831321, 2.49366941,
        2.21259999],
       [1.33875424, 0.11112472, 2.56705619, 2.03577972, 2.86007881,
        3.09934107]])
Coordinates:
  * time      (time) datetime64[ns] 32B 2016-08-01 2016-09-01 ... 2016-11-01
  * location  (location) int64 48B 1000 2800 3000 5500 9500 9600
Attributes:
    standard_name:  Monthly Discharge
    units:          m^3/s

Create an xarray.Dataset and check the format to insure everything is correct

In [10]:
metdata_input = xr.merge([Monthly_Temp, Monthly_Precip, Monthly_Q])
check_format(metdata_input)

All necessary variables exist and are labeled properly.
All necessary coordinates exist and are labeled properly


Here is the final dataset that will be used with the model. 

In [11]:
metdata_input

<xarray.Dataset> Size: 1kB
Dimensions:       (location: 6, time: 6)
Coordinates:
  * location      (location) int64 48B 1000 2800 3000 5500 9500 9600
  * time          (time) datetime64[ns] 48B 2016-01-01 2016-02-01 ... 2016-11-01
    lat           (location) float64 48B 46.7 46.82 46.78 46.67 46.75 46.75
Data variables:
    mo_temp       (time, location) float64 288B -3.117 -2.703 -2.736 ... nan nan
    mo_precip     (time, location) float64 288B 2.211e+10 6.324e+09 ... nan nan
    mo_discharge  (time, location) float64 288B nan nan nan ... 2.036 2.86 3.099
Attributes:
    standard_name:  Monthly_Temperature
    units:          Celcius

## Examples of Code from Above Functions (for refrence only)

<ins>The following code is identical to code found in the above fuctions or of earlier approchs to pulling, calculating area weighted volumes or means of polygons. The following cells do not need to be run in this example</ins>

Below is the code used in the 'get_gridmet_for_polygons()' and 'get_gridmet_for_polygons_with_xvec()' functions. They are included for testing or troubleshooting if needed. The third example shows code similar to 'get_gridmet_at_points()' but it has been modified to pull metdata and calculate area weighted volume for precipitation is shown below. It is included as an example of earlier approches.  

In [ ]:
# Give arguments for upcoming functions
in_geom = exres
gdf_index_col = "gageID"
start='2016-08-01'
end='2016-11-30'
crs = 4326

### get_gridmet_for_polygons() 

In [ ]:
var_list = []
for p in GRIDMET_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable=p, start=start, end=end,
                    bbox=BBox(bnds[0] - 0.5, bnds[2] + 0.5, bnds[3] + 0.5, bnds[1] - 0.5))
    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]

    if p == 'pr':
        vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, gdf_index_col)
        vol_xds = vol_xds.drop('area')
    else:
        # This is done for each output of the above code
        in_geom["gageID"] = in_geom.gageID.astype(int) 
        gmet_input = gmet_input.rio.write_crs(input_crs=4326).rio.clip(in_geom.geometry.values, in_geom.crs)
        gmet_input.name = p

        # Added coords previously not needed in package example
        grid_out = make_geocube(vector_data=in_geom, measurements=["gageID"], like=gmet_input).set_coords('gageID')

        for date in range(0, len(gmet_input.time.values)):
            gmet_ts = gmet_input[date,:,:]
            grid_ts = grid_out
            
            grid_ts[p] = (grid_out.dims, gmet_ts.values, gmet_ts.attrs, gmet_ts.encoding)
            grid_ts = grid_out.drop("spatial_ref").groupby(grid_out.gageID).mean()

            xda = grid_ts[p]
            xda = xda.expand_dims({"time": 1}).assign_coords(time=('time', [gmet_ts.time.values]))
            var_list.append(xda)
    xds = xr.merge(var_list)

# Add lat coordinate and make match order of xds locations
lat_df = pd.DataFrame((in_geom.geometry.bounds['miny'] + in_geom.geometry.bounds['maxy']) / 2).set_index(in_geom.gageID)
lat_df = lat_df.reindex(list(xds.gageID.values.astype(int)))
    
xds = xr.Dataset(
    {
        "max_temp": (['time', 'location'], xds["tmmx"].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'}),
        "min_temp": (['time', 'location'], xds["tmmn"].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], list(lat_df.iloc[:,0]), {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "location": (['location'], xds['gageID'].values.astype(int), {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}), # Keep the order of xds
        "time": xds['time'].values
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

if 'pr' in GRIDMET_PARAMS:
    output = xr.merge([xds, vol_xds])# vol_xds reorders to match xds
else: 
    output = xds

output

### get_gridmet_for_polygons_with_xvec()

In [ ]:
gmet_list = []
for p in GRIDMET_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable=p, start=start, end=end,
                    bbox=BBox(bnds[0] - 0.5, bnds[2] + 0.5, bnds[3] + 0.5, bnds[1] - 0.5))
    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]
    if p == 'pr':
        vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, gdf_index_col)
        vol_xds = vol_xds.transpose('location', 'time').drop('area')
    else:
        gmet_list.append(gmet_input)
ds = xr.merge(gmet_list)
ds = ds.xvec.zonal_stats(geometry=in_geom.geometry, x_coords="lon", y_coords="lat", stats="mean", method="iterate")

# Add coords
## Make and reorder the location ID coordinates
loc_df = pd.DataFrame(in_geom.gageID).set_index(in_geom.geometry)
loc_df = loc_df.reindex(in_geom.geometry.values)

## Calculate and reorder lat coordinates
lat_df = pd.DataFrame((in_geom.geometry.bounds['miny'] + in_geom.geometry.bounds['maxy']) / 2).set_index(in_geom.geometry)
lat_df = lat_df.reindex(in_geom.geometry.values)

xds = xr.Dataset(
    {
        "max_temp": (['location', 'time'], ds['daily_maximum_temperature'].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'}),
        "min_temp": (['location', 'time'], ds['daily_minimum_temperature'].values, {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], list(lat_df.iloc[:,0]), {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "location": (['location'], list(loc_df.iloc[:,0]), {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}),
        "time": ds['time'].values
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

if 'pr' in GRIDMET_PARAMS:
    output = xr.merge([xds, vol_xds])
else: 
    output = xds

output


### Updated and archived get_gridmet_at_points()

In [ ]:
# get_gridmet_at_points()
in_geom = exres
gdf_index_col = "gageID"
start='2016-01-01'
end='2016-01-05'
crs = 4326

# overwrite configuration to run initial approch for pulling gridmet data and performing zonal stats
GRIDMET_PARAMS = ['tmmn', 'tmmx']
VOL_PARAMS = ['pr']


if gdf_index_col is not None:
    ixcol = gdf_index_col
else:
    in_geom['ixcol'] = in_geom.index
    ixcol = 'ixcol'

location_ids = in_geom[ixcol].to_list()

if (in_geom.geometry.geom_type == 'Point').all():
    coords = list(zip(in_geom.geometry.x, in_geom.geometry.y))
elif (in_geom.geometry.geom_type == 'Polygon').all():
    coords = list(zip(in_geom.geometry.centroid.x, in_geom.geometry.centroid.y))
else:
    coords = None
    raise ValueError("Mixed geometry types were found in the input GeoDataFrame. Mixed Geometry is not supported.")

loc_lat = []
loc_lon = []
loc_elev = py3dep.elevation_bycoords(coords, crs=crs)  # only 4326 or NAD83 works with py3dep

if isinstance(loc_elev, list):
    loc_elev = loc_elev
else:
    loc_elev = [loc_elev]

loc_gdf = in_geom[['{0}'.format(ixcol), 'geometry']]

print("Retrieving GridMET cells...")
gmt_cells = get_gridmet_cells(loc_gdf)
unq_cells = gmt_cells['cell_id'].unique()
print("{0} unique GridMET cells found for {1} input features.".format(len(unq_cells), len(loc_gdf[ixcol])))

gmt_cntrs = gmt_cells.drop_duplicates(subset='cell_id').centroid

# Parameters retrieved to be averaged over watershed area here
tmmn = []
tmmx = []

cdsets = {}
print("Fetching GridMET data for unique cells...")
for cell in tqdm(unq_cells, desc='Cells'):
    clon = gmt_cntrs[cell].x
    clat = gmt_cntrs[cell].y
    datasets = []
    for p in GRIDMET_PARAMS:
        s = start
        e = end
        ds = GridMet(p, start=s, end=e, lat=clat, lon=clon).get_point_timeseries()
        datasets.append(ds)
    cdsets[cell] = datasets

# Parameters retried to be converted wot weighted volumes here
if len(VOL_PARAMS)> 1:
    raise ValueError("GRIDtools.grid_area_weighted_volume() is only compatible with the precip parameter")

# volparam_list = []
for p in VOL_PARAMS:
    bnds = in_geom.total_bounds
    gmet = GridMet(variable= p, start=start, end=end, bbox=BBox(bnds[0]-0.5, bnds[2]+0.5, bnds[3]+0.5, bnds[1]-0.5))

    gmet = gmet.subset_nc(return_array=True)
    gmet_input = gmet[list(gmet.data_vars)[0]]
    vol_xds = gt.grid_area_weighted_volume(gmet_input, in_geom, 'gageID')
    # volparam_list.append(vol_xds)
# xr.merge(volparam_list)

for i in range(len(coords)):
    c = coords[i]
    loc = location_ids[i]
    gmtcell_ids = gmt_cells[gmt_cells[ixcol] == loc]
    lon, lat = c
    loc_lat.append(lat)
    loc_lon.append(lon)


    if len(gmtcell_ids.index) > 1:

        tmmnm = []
        tmmxm = []

        for cid in gmtcell_ids['cell_id']:
            dset = cdsets[cid]

            tmmnm.append(dset[GRIDMET_PARAMS.index('tmmn')])
            tmmxm.append(dset[GRIDMET_PARAMS.index('tmmx')])

        tmmnm_d = pd.concat(tmmnm)
        tmmxm_d = pd.concat(tmmxm)

        tmmn.append(tmmnm_d.groupby(tmmnm_d.index).mean())
        tmmx.append(tmmxm_d.groupby(tmmxm_d.index).mean())

    else:
        dset = cdsets[gmtcell_ids['cell_id'].values[0]]
        tmmn.append(dset[GRIDMET_PARAMS.index('tmmn')])
        tmmx.append(dset[GRIDMET_PARAMS.index('tmmx')])

mean_xds = xr.Dataset(
    {
        "min_temp": (['time', 'location'], pd.concat(tmmn, axis=1), {'standard_name': 'Minimum Temperature',
                                                                    'units': 'Kelvin'}),
        "max_temp": (['time', 'location'], pd.concat(tmmx, axis=1), {'standard_name': 'Maximum Temperature',
                                                                    'units': 'Kelvin'})
    },
    coords={
        "lat": (['location'], loc_lat, {'standard_name': 'latitude',
                                        'long_name': 'location_latitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "lon": (['location'], loc_lon, {'standard_name': 'longitude',
                                        'long_name': 'location_longitude',
                                        'units': 'degrees',
                                        'crs': '4326'}),
        "elev": (['location'], loc_elev, {'standard_name': 'elevation',
                                        'long_name': 'location_elevation',
                                        'units': 'meters'}),
        "location": (['location'], location_ids, {'long_name': 'location_identifier',
                                        'cf_role': 'timeseries_id'}),
        "time": tmmn[0].index
    },
    attrs={
        "featureType": 'timeSeries',
    }
)

xr.merge([mean_xds, vol_xds])